* * *

**1\. 多头注意力的基本流程**
------------------

### **(1) 输入和投影**

假设我们有一个输入张量 $X$：

*   **形状：** $(batch\_size, seq\_len, hidden\_dim)$

我们使用 **三个线性变换**（`nn.Linear`）得到：

*   **查询（Query, Q）**
*   **键（Key, K）**
*   **值（Value, V）**

这些变换通常共享 `hidden_dim` 作为输入和输出的维度：

$$Q, K, V = X W_Q, X W_K, X W_V$$

其中：

*   $W_Q, W_K, W_V$ 是线性变换矩阵，形状为 $(hidden\_dim, hidden\_dim)$。

### **(2) 切分成多个头**

为了让注意力机制能够从不同的子空间学习不同的信息，我们引入 `head_num` 个头，每个头的维度是 `head_dim`，并将 `hidden_dim` 平均分成 `head_num` 份：

$$head\_dim = \frac{hidden\_dim}{head\_num}$$

然后，我们将 $Q, K, V$ **reshape** 为：

$$(batch\_size, seq\_len, head\_num, head\_dim)$$

然后进行维度转换，使其变为：

$$(batch\_size, head\_num, seq\_len, head\_dim)$$

这样，每个头就可以单独计算自注意力。

### **(3) 计算每个头的注意力**

对于每个头 $i$，计算：

$$\text{Attention}_i = \text{softmax} \left( \frac{Q_i K_i^T}{\sqrt{head\_dim}} \right) V_i$$

其中：

*   $Q_i, K_i, V_i$ 的形状均为 $(batch\_size, seq\_len, head\_dim)$。

### **(4) 多头结果合并**

计算完所有头的注意力后，我们需要将多个头的结果拼接回原始维度：

$$\text{MultiHeadOutput} = \text{Concat}(\text{Attention}_1, ..., \text{Attention}_h)$$

这样拼接后，形状变为：

$$(batch\_size, seq\_len, head\_num \times head\_dim)$$

为了保证最终的输出维度仍然是 `hidden_dim`，**必须满足**：

$$head\_num \times head\_dim = hidden\_dim$$

* * *

**2\. 为什么要保证 `head_num * head_dim = hidden_dim`？**
--------------------------------------------------

1.  **保证输入和输出的维度一致**
    
    *   这样 Residual Connection（残差连接）才能正确相加： 
        $$X + \text{MultiHeadOutput}$$
        
    *   如果 `MultiHeadOutput` 维度不匹配 `hidden_dim`，残差连接无法进行。
2.  **确保计算效率**
    
    *   这样可以直接拼接多个头的输出，而不需要额外的维度变换。
3.  **让每个注意力头关注不同的信息**
    
    *   例如，某个头可能关注语义信息，另一个头可能关注位置关系。

* * *

In [4]:
import math
import torch
import torch.nn as nn

class MutiHeadSelfAttentionFormal(nn.Module):
    def __init__(self, hidden_dim, head_num, attention_dropout=0.1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.head_num = head_num
        self.head_dim  = hidden_dim // head_num #head_num * head_dim = hidden_dim
        self.q_proj = nn.Linear(hidden_dim,hidden_dim)
        self.k_proj = nn.Linear(hidden_dim,hidden_dim)
        self.v_proj = nn.Linear(hidden_dim,hidden_dim)
        self.out_proj = nn.Linear(hidden_dim,hidden_dim)

        self.attention_dropout = nn.Dropout(attention_dropout)
    
    def forward(self,X,attention_mask=None):
        #X shape (batch,seq,hidden_dim)
        batch,seq_len,_ =X.size()
        Q = self.q_proj(X)
        K = self.k_proj(X)
        v = self.v_proj(X) #shape(b,seq,hidden_dim)
        #(b,seq,hidden_dim) --> (b,head_num,seq,head_dim)
        #因此需要将hidden_dim拆成head_dim*head_num
        q_state = Q.view(batch,seq_len,self.head_num,self.head_dim).transpose(1,2)
        k_state = K.view(batch,seq_len,self.head_num,self.head_dim).transpose(1,2)
        v_state = v.view(batch,seq_len,self.head_num,self.head_dim).transpose(1,2)
        #attention_weight shape (b,head_num,s,s)
        attention_weight = torch.matmul( 
            q_state,k_state.transpose(-1,-2)  #(b,head_num,seq,head_dim)  --> (b,head_num,head_dim,seq)
        ) / torch.sqrt(torch.tensor(self.head_dim,device=X.device))

        if attention_mask is not None:
            attention_weight = attention_weight.masked_fill(
                attention_mask == 0,
                float('-inf')
            )
        print(f"attention_weight is \n {attention_weight}")
        attention_weight = torch.softmax(attention_weight,dim=-1)
        attention_weight = self.attention_dropout(attention_weight)
        output_mid = torch.matmul(attention_weight,v_state) #(b,head_num,seq,head_dim)
        output_mid = output_mid.transpose(1,2).contiguous()
        output_mid = output_mid.view(batch,seq_len,-1) #-1自动填充为hidden_dim维度
        output = self.out_proj(output_mid)
        return output
    

attention_mask = (
    torch.tensor(
        [
            [0,1],
            [0,0],
            [1,0]
        ]
    ).unsqueeze(1).unsqueeze(2).expand(3,8,2,2)
)

X = torch.rand(3,2,128)
net  = MutiHeadSelfAttentionFormal(128,8) #head_dim = 16
net(X,attention_mask)






attention_weight is 
 tensor([[[[   -inf, -0.2542],
          [   -inf, -0.1895]],

         [[   -inf, -0.1817],
          [   -inf, -0.2573]],

         [[   -inf,  0.0769],
          [   -inf,  0.0249]],

         [[   -inf,  0.2377],
          [   -inf,  0.1909]],

         [[   -inf,  0.0548],
          [   -inf,  0.0169]],

         [[   -inf, -0.0766],
          [   -inf, -0.1476]],

         [[   -inf, -0.1451],
          [   -inf, -0.1433]],

         [[   -inf, -0.1044],
          [   -inf, -0.3028]]],


        [[[   -inf,    -inf],
          [   -inf,    -inf]],

         [[   -inf,    -inf],
          [   -inf,    -inf]],

         [[   -inf,    -inf],
          [   -inf,    -inf]],

         [[   -inf,    -inf],
          [   -inf,    -inf]],

         [[   -inf,    -inf],
          [   -inf,    -inf]],

         [[   -inf,    -inf],
          [   -inf,    -inf]],

         [[   -inf,    -inf],
          [   -inf,    -inf]],

         [[   -inf,    -inf],
          [   -i

tensor([[[-4.7794e-02, -3.8403e-02, -1.7401e-01,  1.8290e-01, -7.4299e-02,
           1.3316e-01, -1.9407e-01, -3.7210e-01,  2.2652e-01,  9.2041e-02,
           1.8681e-01,  7.9244e-02,  4.3188e-01,  1.9363e-01, -1.8323e-01,
           3.4159e-01, -1.6718e-01, -5.3080e-02,  3.9149e-02,  3.1558e-02,
           3.2350e-01, -3.9447e-01,  3.0021e-01,  2.3686e-01, -1.8960e-01,
           2.5181e-01, -1.1322e-02, -3.7456e-01,  4.6022e-02, -1.1317e-01,
          -7.0249e-03, -2.3937e-01, -2.7386e-01,  1.2554e-01, -1.5031e-01,
          -1.8619e-02, -1.9030e-01,  1.6299e-01,  2.8923e-01, -1.1642e-01,
          -8.8469e-02, -3.7217e-01, -1.5708e-01, -1.4586e-01,  3.7437e-01,
           3.9479e-01,  9.8606e-02,  2.1099e-01, -3.7894e-03, -7.2547e-02,
          -5.6828e-02,  1.6051e-01,  1.7420e-01, -7.9083e-02,  1.2469e-01,
          -6.9784e-02, -4.9505e-02,  1.6026e-01,  3.1288e-01,  3.5575e-01,
           5.4081e-02, -2.1109e-01, -3.7861e-01, -3.0217e-01, -2.5117e-01,
           2.2149e-01,  3